# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Import collected city weather as DataFrame
city_weather = pd.read_csv("../WeatherPy/Resources/Sample_Weather_Data.csv")
# Clean cities with no weather data
city_weather_clean = city_weather.dropna()
city_weather_clean.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,"mahebourg, mu",-20.4081,57.7000,80.60,83.0,40.0,3.44,MU,1.612413e+09
2,"hithadhoo, mv",-0.6000,73.0833,82.98,72.0,59.0,7.05,MV,1.612413e+09
3,"vila franca do campo, pt",37.7167,-25.4333,51.80,71.0,40.0,12.66,PT,1.612413e+09
4,"busselton, au",-33.6500,115.3333,73.00,42.0,0.0,4.00,AU,1.612413e+09
5,"manta, ec",-0.9500,-80.7333,76.96,87.0,66.0,8.19,EC,1.612413e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_clean[["Lat", "Lng"]].astype(float)

# Store associated humidity values for weights
humidity = city_weather_clean["Humidity"].astype(float)


In [18]:
# Customize the size of the map figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create the figure with layout parameters
fig = gmaps.figure(layout=figure_layout)

# Create the heat layer with location data and humidity weights 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

# Add the layer to the figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Refine the weather results to readings that would seem fitting for a vacation
vacation_weather = city_weather_clean.loc[(city_weather_clean["Max Temp"] < 80) &
                                          (city_weather_clean["Max Temp"] > 70) &
                                          (city_weather_clean["Humidity"] < 80) & 
                                          (city_weather_clean["Wind Speed"] < 10) &
                                          (city_weather_clean["Cloudiness"] == 0)]

# Check the count to see if it's a reasonable number
vacation_weather.count()

City          15
Lat           15
Lng           15
Max Temp      15
Humidity      15
Cloudiness    15
Wind Speed    15
Country       15
Date          15
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = pd.DataFrame(columns=['Lat', 'Lng', 'Hotel Name', 'City', 'Country'])
hotel_df

,Lat,Lng,Hotel Name,City,Country


In [21]:
# set default for target coordinates
target_coordinates = ""
# set radius of 5000 meters
target_radius = 5000
# set the type of results for Google to search for
target_type = "lodging"

# Iterate through vacation data for coordinates and create hotel dataframe
for index, row in vacation_weather.iterrows():
    
    # geocoordinates
    lat = vacation_weather.loc[index, "Lat"]
    lng = vacation_weather.loc[index, "Lng"]
    
    # Populate what is already known into the new hotel dataframe
    target_coordinates = f'{lat}, {lng}'
    hotel_df.loc[index, "Lat"] = lat
    hotel_df.loc[index, "Lng"] = lng
    hotel_df.loc[index, "City"] = vacation_weather.loc[index, "City"]
    hotel_df.loc[index, "Country"] = vacation_weather.loc[index, "Country"]
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # base Google Places url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    try:
        # run a request using assembled params dictionary
        response = requests.get(base_url, params=params)

        places_data = response.json()
    
        hotel_df.loc[index, "Hotel Name"] = places_data["results"][0]["name"]
            
    except:
        hotel_df.loc[index, "Hotel Name"] = "Unknown"
        
hotel_df

,Lat,Lng,Hotel Name,City,Country
4,-33.65,115.333,Observatory Guest House,"busselton, au",AU
25,19.3003,97.9685,Baiyoke Chalet Hotel,"mae hong son, th",TH
100,24.6833,92.5667,Hotel Purnolaxmi,"hailakandi, in",IN
164,25.8333,93.4333,Hotel Kamakhya,"diphu, in",IN
183,19.2667,84.9167,Swosti Palm Resort,"gopalpur, in",IN
184,12.35,32.1833,Unknown,"marabba, sd",SD
197,23.9,79.2,Ramesh Walding Workshop,"patharia, in",IN
199,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel","mecca, sa",SA
328,16.3333,-15,Unknown,"saint-louis, sn",SN
374,20.15,92.9,Hotel Memory,"akyab, mm",MM


In [22]:
# Starter code
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…